In [13]:
import mesa
import seaborn as sns
import numpy as np
import pandas as pd
import random as rnd

In [14]:
class Agent(mesa.Agent):
    def __init__(self, id, model):
        super().__init__(id, model)
        self.base_dissatisfaction = 20
        self.dissatisfaction = rnd.randint(0, self.base_dissatisfaction)
        self.close_friends = np.array([])
        self.friends = np.array([])
        self.network = np.array([])
        self.id = id
        
    def add_close_friend(self,close_friend):
        self.close_friends = np.append(self.close_friends,close_friend)
        
    def add_friend(self,friend):
        self.friends = np.append(self.friends,friend)
        
    def add_to_network(self,network_friend):
        self.network = np.append(self.network,network_friend)

    def step(self):
        close_friendgroup = np.array([agents[0+i*4],agents[1+i*4],agents[2+i*4],agents[3+i*4]])
        agents[(0+i*4)].add_close_friend(np.array([agents[1+i*4],agents[2+i*4],agents[3+i*4]]))
        agents[(1+i*4)].add_close_friend(np.array([agents[0+i*4],agents[2+i*4],agents[3+i*4]]))
        agents[(2+i*4)].add_close_friend(np.array([agents[0+i*4],agents[1+i*4],agents[3+i*4]]))
        agents[(3+i*4)].add_close_friend(np.array([agents[0+i*4],agents[1+i*4],agents[2+i*4]]))
        print(f"Hi, I am an agent, you can call me {str(self.dissatisfaction)}.")

class Model(mesa.Model):
    def __init__(self, N):
        self.num_agents = N
        self.schedule = mesa.time.RandomActivation(self)
        for i in range(self.num_agents):
            a = Agent(i, self)
            self.schedule.add(a)

    def step(self):
        self.schedule.step()

In [15]:
starter_model = Model(10)
starter_model.step()

Hi, I am an agent, you can call me 9.
Hi, I am an agent, you can call me 5.
Hi, I am an agent, you can call me 4.
Hi, I am an agent, you can call me 11.
Hi, I am an agent, you can call me 1.
Hi, I am an agent, you can call me 10.
Hi, I am an agent, you can call me 15.
Hi, I am an agent, you can call me 13.
Hi, I am an agent, you can call me 5.
Hi, I am an agent, you can call me 7.
